# Import packages

In [1]:
import os
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray as rxr
import rasterio as rio
import matplotlib.pyplot as plt
import contextily as cx
import matplotlib.dates as mdates
from datetime import datetime
from hydromt_wflow import WflowModel
from hydromt.config import configread
from hydromt.log import setuplog

ModuleNotFoundError: No module named 'hydromt_wflow'

# Build model

In [ ]:
# Read template config model file
model_file = configread('/Users/aprida/Documents/Consulting/Private_sector/Keolis/Model_Alvaro/wflow_build.yml', abs_path=True)

# Set up model root and data parameters
root = str(model_file['root']) # Model name
data_libs = str(model_file['data_libs']) # Data catalog path
logger = setuplog(root, log_level = 10) # Show messages when executing model operations

# Build model
wf = WflowModel(root=root, mode='r', data_libs=data_libs) # Initialize model class -> Create model folder

# Setup config

In [4]:
import rioxarray as rxr
fn = '/Users/aprida/.hydromt_data/artifact_data/v0.0.8/era5.nc'
ds = rxr.open_rasterio(fn)
ds['spatial_ref']

<xarray.DataArray 'spatial_ref' ()>
array(0)
Coordinates:
    spatial_ref  int64 0
Attributes:
    GeoTransform:  11.625 0.25 0.0 46.875 0.0 -0.25

In [ ]:
start_time = model_file['setup_config']['starttime']
end_time = model_file['setup_config']['endtime']
wf.setup_config(starttime=start_time, endtime=end_time)

# Setup grid

In [ ]:
# Define grid inputs
region = model_file['region']
hydrography_fn = model_file['setup_basemaps']['hydrography_fn']
res = model_file['setup_basemaps']['res']
crs = model_file['crs']

# Generate topography static layers and grid
wf.setup_basemaps(region={'subbasin':[4.83143, 46.29591], 'uparea': 10}, hydrography_fn=hydrography_fn, res=res)

# Setup rivers

In [ ]:
# Define river inputs
river_geom_fn = model_file['setup_rivers']['river_geom_fn']

# Generate river WFlow layers
wf.setup_rivers(hydrography_fn=hydrography_fn, river_geom_fn=river_geom_fn)

# Setup LULC

In [ ]:
# Define LULC inputs
lulc_fn = model_file['setup_lulcmaps']['lulc_fn']
lulc_mapping_fn = model_file['setup_lulcmaps']['lulc_mapping_fn']
lulc_vars = model_file['setup_lulcmaps']['lulc_vars']

# Generate LULC WFlow layers
wf.setup_lulcmaps(lulc_fn=lulc_fn, lulc_mapping_fn=lulc_mapping_fn, lulc_vars=lulc_vars)

# Setup LAI

In [ ]:
# Define LAI inputs
lai_fn = model_file['setup_laimaps']['lai_fn']

# Generate LAI WFlow layers
wf.setup_laimaps(lai_fn=lai_fn)

# Setup Soil Maps

In [ ]:
# Define Soil inputs
soil_fn = model_file['setup_soilmaps']['soil_fn']

# Generate Soil WFlow layers
wf.setup_soilmaps(soil_fn=soil_fn)

# Setup Outlets and Gauges

In [ ]:
# Outlets
river_only = model_file['setup_outlets']['river_only']
toml_output = model_file['setup_outlets']['toml_output']
gauge_toml_header = model_file['setup_outlets']['gauge_toml_header']

wf.setup_outlets(river_only=True, toml_output='csv', gauge_toml_header=['Q']) # Setup a location to store results at the outlet of the model

# Gauges / Observation points
gauges_fn = model_file['setup_gauges']['gauges_fn']
snap_to_river = model_file['setup_gauges']['snap_to_river']

wf.setup_gauges(gauges_fn=gauges_fn, snap_to_river=snap_to_river, derive_subcatch=True) # Setup locations to store results based on a .csv file with lat/lons

# Setup Precipitation, Temperature and ETP Forcing

In [ ]:
t_start = datetime.now()

# Prepare precipitation input
precip_fn = model_file['setup_precip_forcing']['precip_fn']
precip_fn = str(precip_fn)
skip_pet = model_file['setup_temp_pet_forcing']['skip_pet']

# Generate precipitation WFlow layer
wf.setup_precip_forcing(precip_fn=precip_fn)
wf.setup_temp_pet_forcing(temp_pet_fn=precip_fn, skip_pet=skip_pet)
# wf.write_forcing()

print(datetime.now() - t_start)

# Setup constant parameters

In [ ]:
ctt_parameters = {
    "KsatHorFrac": 100,
    "Cfmax": 3.75653,
    "cf_soil": 0.038,
    "EoverR": 0.11,
    "InfiltCapPath": 5,
    "InfiltCapSoil": 600, # In mm/dt, 600 mm/day is extremely high
    "MaxLeakage": 0,
    "rootdistpar": -500,
    "TT": 0,
    "TTI": 2,
    "TTM": 0,
    "WHC": 0.1,
    "G_Cfmax": 5.3,
    "G_SIfrac": 0.002,
    "G_TT": 1.3
}
wf.setup_constant_pars(**ctt_parameters)

# Write inputs

In [ ]:
t_start = datetime.now()

wf.write()

print(datetime.now() - t_start)

# Run model

In [ ]:
runpath = f"""
julia -e 'using Wflow; Wflow.run()' '{root}/wflow_sbm.toml'
"""

os.system(runpath)

# Outputs

In [ ]:
# Read template config model file
model_file = configread('/Users/aprida/Documents/Consulting/Private_sector/Keolis/Model_Alvaro/wflow_build.yml', abs_path=True)

# Set up model root and data parameters
root = str(model_file['root']) # Model name
data_libs = str(model_file['data_libs']) # Data catalog path
logger = setuplog(root, log_level = 10) # Show messages when executing model operations

# Build model
wf = WflowModel(root=root, mode='r', data_libs=data_libs) # Initialize model class -> Create model folder

In [ ]:
# Read simulation output
main_gauges = wf.staticgeoms['gauges_obs-points-hydrology']
main_gauges = main_gauges.reset_index(drop=True)
output_sim = os.path.join(root, 'run_default/output.csv')
df_sim = pd.read_csv(output_sim, index_col='time')
df_sim.index = pd.to_datetime(df_sim.index)

# Read observations
df_q_obs = pd.read_csv('/Users/aprida/Documents/Consulting/Private_sector/Keolis/Model_Alvaro/Inputs/historical_ts_discharge_stations.csv',
                       index_col='date_obs_elab',
                      parse_dates=['date_obs_elab'])[['code_station', 'resultat_obs_elab']]
# df_q_obs = df_q_obs.loc[df_sim_q.index.min():df_sim_q.index.max()]

# Plot simulation vs observation discharges
df_sim_q = df_sim.filter(like='Q')

In [ ]:
# Read simulation output
main_gauges = wf.staticgeoms['gauges_obs-points-hydrology']
main_gauges = main_gauges.reset_index(drop=True)
output_sim = os.path.join(root, 'run_default/output.csv')
df_sim = pd.read_csv(output_sim, index_col='time', parse_dates=['time'])

# Read observations
df_q_obs = pd.read_csv('/Users/aprida/Documents/Consulting/Private_sector/Keolis/Model_Alvaro/Inputs/historical_ts_discharge_stations.csv',
                       index_col='date_obs_elab',
                      parse_dates=['date_obs_elab'])[['code_station', 'resultat_obs_elab']]
# df_q_obs = df_q_obs.loc[df_sim_q.index.min():df_sim_q.index.max()]

# Plot simulation vs observation discharges
df_sim_q = df_sim.filter(like='Q')
df_sim_q.columns = [x.split('_')[-1] for x in df_sim_q.columns]

plt.rcParams['font.size'] = 13

for gauge in df_sim_q.columns[1:]:
    
    fig, ax = plt.subplots()
    
    df_sim_q[gauge].plot(ax=ax, label='Simulation')
    
    code_station = main_gauges[main_gauges['id']==int(gauge)].reset_index(drop=True)['code_station'][0]
    
    df_q_obs[df_q_obs['code_station']==code_station].loc[df_sim_q.index.min():df_sim_q.index.max()]['resultat_obs_elab'].plot(ax=ax, label='Observation')
    ax.legend()
    ax.set_xlabel('')
    ax.set_ylabel('Discharge (cms)')
    half_year_locator = mdates.DayLocator(interval=15)
    ax.xaxis.set_major_locator(half_year_locator)
    ax.set_title(f'Discharge Observation vs Simulation\n{code_station} - ID = {gauge}')
    ax.grid()
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

fig, ax_sg = plt.subplots(figsize=(12,10))

wf.staticgeoms['rivers'].plot(ax=ax_sg, alpha=0.8, color='blue', linewidth=wf.staticgeoms['rivers']['strord'] / 2)
wf.staticgeoms['basins'].boundary.plot(ax=ax_sg, alpha=0.8, edgecolor='green', linewidth=2)
wf.staticgeoms['gauges'].plot(ax=ax_sg, color='red', markersize=100, zorder=4)
cx.add_basemap(ax=ax_sg, crs=4326, source=cx.providers.Esri.WorldTopoMap, attribution=False)
main_gauges.plot(ax=ax_sg, color='red', markersize=100, zorder=4)
ax_sg.set_xlabel('Longitude (degrees)')
ax_sg.set_ylabel('Latitude (degrees)')

for x, y, label in zip(main_gauges.geometry.x, main_gauges.geometry.y, main_gauges.id):
    ax_sg.annotate(label, xy=(x, y), xytext=(7, 7), textcoords="offset points", color='red')

In [ ]:
code_station = 'U430001001'

# Plot simulation vs observation at the outlet

fig, ax = plt.subplots()

df_sim_q['1'].plot(ax=ax, label='Simulation')

# code_station = main_gauges[main_gauges['id']==int(gauge)].reset_index(drop=True)['code_station'][0]

df_q_obs[df_q_obs['code_station']==code_station].loc[df_sim_q.index.min():df_sim_q.index.max()]['resultat_obs_elab'].plot(ax=ax, label='Observation')
ax.legend()
ax.set_xlabel('')
ax.set_ylabel('Discharge (cms)')
half_year_locator = mdates.DayLocator(interval=15)
ax.xaxis.set_major_locator(half_year_locator)
ax.set_title(f'Discharge Observation vs Simulation\n{code_station} - ID = 1')
ax.grid()
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
# Plot precipitation along simulation
df_sim_p = df_sim.filter(like='P')

fig, ax = plt.subplots()

df_sim_p.plot(ax=ax)
ax.legend()
ax.set_xlabel('')
# plt.xticks(rotation='vertical')
ax.set_ylabel('Precipitation (mm)')
half_year_locator = mdates.DayLocator(interval=15)
# ax.xaxis.set_major_locator(half_year_locator)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.grid()